In [2]:
# import tensorflow as tf

# # 첫 예제
# a = tf.constant(5, name="input_a")
# b = tf.constant(3, name="input_b")
# c = tf.multiply(a,b, name="mul_c")
# d = tf.add(a,b, name="add_d")
# e = tf.add(c,d, name="add_e")
# sess = tf.Session()
# sess.run(e)
# writer = tf.summary.FileWriter('./my_graph', sess.graph)
# sess.close()

23

In [16]:
import tensorflow as tf

# Explicitly create a Graph object
graph = tf.Graph()

with graph.as_default():
    with tf.name_scope("variables"):
        # Variable to keep tfack of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")
        
        # Variable that keeps track of the sum of all output values over time:
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="total_output")
        
        # Primary transformation Operations
        with tf.name_scope("transformation"):
            # Separate input layer
            with tf.name_scope("input"):
                # Create input placeholder-takes in a Vector
                a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
                
            # Separate middle layer
            with tf.name_scope("intermediate_layer"):
                b = tf.reduce_prod(a, name="product_b")
                c = tf.reduce_sum(a, name="sum_c")
                
                # Seperate output layer
                with tf.name_scope("output"):
                    output = tf.add(b, c, name="output")
                    
                with tf.name_scope("update"):
                    # Increments the total_output Variable by the latest output
                    update_total = total_output.assign_add(output)
                    
                    # Increments the above 'global_step' Cariable, should be run whenever the graph is run
                    increment_step = global_step.assign_add(1)
            # Summary Operations
            with tf.name_scope("summaries"):
                avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
                
                # Creates summaries for output node
                tf.summary.scalar(name="output_summary",output)
                tf.summary.scalar(name="total_summary", update_total)
                tf.summary.scalar(name="average_summary", avg)
            
            # Global Variables and Operations
            with tf.name_scope("global_ops"):
                # Initializtion Op
                init = tf.initialize_all_variables()
                # Merge all summaries into one Operation
                merged_summaries = tf.merge_all_summaries()
                    
# Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)
# Open a SummaryWriter to save summaries
writer = tf.summary.FileWriter('./improved_graph', graph)

# Initialize Variables
sess.run(init)

def run_graph(input_tensor):
    # Helper function; runs the graph with given input tensor and saves summaries
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)
    
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

# Write the summaries to disk
writer.flush()

# Close the SummaryWriter
writer.close()

# Close the session
sess.close()               

SyntaxError: positional argument follows keyword argument (<ipython-input-16-ba901696ed23>, line 41)